<a href="https://colab.research.google.com/github/alexandergribenchenko/Pycaret_Exploration/blob/main/Acamica_PY_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto 04 - Informe final de carrera**

### (Acámica - DS online 40 - Proyecto 04 - Alexander Ortega)